In [4]:
%load_ext autoreload
%autoreload 2

In [1]:
from scripts.benchmark_functions import *

Creating batches of data for benchmarking

In [2]:
dataset_path = 'datasets/exoromper/'
batch_size = 20
num_batches = 10
batch_compositions = [{'space':10, 'earth':5, 'lens_flare':5} for i in range(num_batches)]
# test_seq, labels, fnames = create_benchmark_seq_batches(dataset_path, batch_size, num_batches, batch_compositions, position_only=True, seed=11)
test_seq, labels, fnames = create_benchmark_seq_batches(dataset_path, batch_size, num_batches, batch_compositions, position_only=True, seed=95)
fnprefix = "seed_95"

Create SCOD model

In [3]:
load_model_path = './best_model/ex_v5_best_model.pt'
dataset_path = 'datasets/exoromper/'
unc_model = create_scod_model(load_model_path, dataset_path, batch_size)

<class 'scod.sketching.sketched_pca.SRFT_SinglePassPCA'>
computing basis
using T = 16


 38%|███▊      | 6/16 [00:04<00:06,  1.44it/s]


Test different flagging algorithms

In [5]:
flaggers_to_test = []
# Naive false
flaggers_to_test += [(lambda x: [False for i in range(len(x))], 'naive_false')]

# Naive true
flaggers_to_test += [(lambda x: [True for i in range(len(x))], 'naive_true')]

# SCOD k-highest threshold
flaggers_to_test += [(lambda x, fl=fl: scod_flagger(x, unc_model, flag_limit=fl), 'scod_'+str(fl)) for fl in range(1,16)]

# Diverse subsampling via coreset
flaggers_to_test += [(lambda x, fl=fl: ds_scod_flagger(x, unc_model, fl), 'diverse_'+str(fl)) for fl in range(1,16)]

# Random
flaggers_to_test += [(lambda x, fl=fl, seed=sd: random_flagger(x, fl, seed=sd), 'rand_'+str(fl)+'_'+str(sd)) for fl in [2,10,15] for sd in [11,95,42,99,13]]


In [6]:
flaggers_to_test

[(<function __main__.<lambda>(x)>, 'naive_false'),
 (<function __main__.<lambda>(x)>, 'naive_true'),
 (<function __main__.<listcomp>.<lambda>(x, fl=1)>, 'scod_1'),
 (<function __main__.<listcomp>.<lambda>(x, fl=2)>, 'scod_2'),
 (<function __main__.<listcomp>.<lambda>(x, fl=3)>, 'scod_3'),
 (<function __main__.<listcomp>.<lambda>(x, fl=4)>, 'scod_4'),
 (<function __main__.<listcomp>.<lambda>(x, fl=5)>, 'scod_5'),
 (<function __main__.<listcomp>.<lambda>(x, fl=6)>, 'scod_6'),
 (<function __main__.<listcomp>.<lambda>(x, fl=7)>, 'scod_7'),
 (<function __main__.<listcomp>.<lambda>(x, fl=8)>, 'scod_8'),
 (<function __main__.<listcomp>.<lambda>(x, fl=9)>, 'scod_9'),
 (<function __main__.<listcomp>.<lambda>(x, fl=10)>, 'scod_10'),
 (<function __main__.<listcomp>.<lambda>(x, fl=11)>, 'scod_11'),
 (<function __main__.<listcomp>.<lambda>(x, fl=12)>, 'scod_12'),
 (<function __main__.<listcomp>.<lambda>(x, fl=13)>, 'scod_13'),
 (<function __main__.<listcomp>.<lambda>(x, fl=14)>, 'scod_14'),
 (<func

In [14]:
costs, accs, accs_std = eval_flaggers(flaggers_to_test, load_model_path, test_seq, labels, indiv=False, fnprefix = fnprefix)

Evaluating algorithm  naive_false
Saving cost/acc data for  naive_false  to file  seed_95_naive_false_cost_acc.npz
Saving mean cost/ mean acc/ stddev cost of algorithms so far to file  seed_95_cost_acc_20220709-000604.npz
Evaluating algorithm  naive_true


/home/somrita/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([1, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Saving cost/acc data for  naive_true  to file  seed_95_naive_true_cost_acc.npz
Saving mean cost/ mean acc/ stddev cost of algorithms so far to file  seed_95_cost_acc_20220709-000943.npz
Evaluating algorithm  scod_1
Saving cost/acc data for  scod_1  to file  seed_95_scod_1_cost_acc.npz
Saving mean cost/ mean acc/ stddev cost of algorithms so far to file  seed_95_cost_acc_20220709-000958.npz
Evaluating algorithm  scod_2
Saving cost/acc data for  scod_2  to file  seed_95_scod_2_cost_acc.npz
Saving mean cost/ mean acc/ stddev cost of algorithms so far to file  seed_95_cost_acc_20220709-001024.npz
Evaluating algorithm  scod_3
Saving cost/acc data for  scod_3  to file  seed_95_scod_3_cost_acc.npz
Saving mean cost/ mean acc/ stddev cost of algorithms so far to file  seed_95_cost_acc_20220709-001109.npz
Evaluating algorithm  scod_4
Saving cost/acc data for  scod_4  to file  seed_95_scod_4_cost_acc.npz
Saving mean cost/ mean acc/ stddev cost of algorithms so far to file  seed_95_cost_acc_202207

: 

In [9]:
# costs_from_file, accs_from_file = load_costs_accs_from_file('cost_acc_20220708-065847.npz')
# costs_from_file, accs_from_file, stddevs_from_file = load_costs_accs_from_file('cost_acc_20220708-074420.npz')
# costs_from_file, accs_from_file, stddevs_from_file = load_costs_accs_from_file('cost_acc_20220708-173917.npz')
# costs_mean_from_file, accs_mean_from_file, accs_stddevs_from_file = load_costs_accs_from_file('cost_acc_20220708-221330.npz')
costs_mean_from_file, accs_mean_from_file, accs_stddevs_from_file = load_costs_accs_from_file('cost_acc_20220709-000002.npz')

print(costs_mean_from_file)
print(accs_mean_from_file)
print(accs_stddevs_from_file)

{'naive_false': 0.0, 'naive_true': 100.5, 'scod_1': 4.87, 'scod_2': 9.615, 'scod_3': 14.35, 'scod_4': 19.225, 'scod_5': 24.015, 'scod_6': 28.78, 'scod_7': 33.68, 'scod_8': 38.705, 'scod_9': 43.695, 'scod_10': 48.855, 'scod_11': 54.04, 'scod_12': 59.125, 'scod_13': 64.195, 'scod_14': 69.275, 'scod_15': 74.455, 'diverse_1': 5.115, 'diverse_2': 9.885, 'diverse_3': 14.845, 'diverse_4': 19.825, 'diverse_5': 24.215, 'diverse_6': 29.185, 'diverse_7': 33.26, 'diverse_8': 38.275, 'diverse_9': 42.105, 'diverse_10': 44.88, 'diverse_11': 45.85, 'diverse_12': 48.055, 'diverse_13': 51.33, 'diverse_14': 54.05, 'diverse_15': 55.865, 'rand_2_11': 9.85, 'rand_2_95': 9.85, 'rand_2_42': 9.85, 'rand_2_99': 9.85, 'rand_2_13': 9.85, 'rand_10_11': 50.7, 'rand_10_95': 50.7, 'rand_10_42': 50.7, 'rand_10_99': 50.7, 'rand_10_13': 50.7, 'rand_15_11': 76.45, 'rand_15_95': 76.45, 'rand_15_42': 76.45, 'rand_15_99': 76.45, 'rand_15_13': 76.45}
{'naive_false': -28.988775, 'naive_true': -28.752216, 'scod_1': -28.894747,

In [23]:
# cost_vec, acc_vec = load_costs_accs_alg_specific_from_file('_naive_false_cost_acc.npz')
# print(cost_vec)
# print(acc_vec)